In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
dir = "/content/drive/MyDrive/Colab Notebooks/git/implementation/DL/Transformer"
data_dir = dir + "/data"
model_dir = dir + "/model"

train_files = [(data_dir+f"/train{i}.xlsx") for i in range(1, 5)]
merged_train_file = data_dir + "/merged_train.xlsx"
test_file = data_dir + "/test.xlsx"

In [5]:
import pandas as pd

train_dfs = []
for fname in train_files:
    train_dfs.append(pd.read_excel(fname))

train_dfs[0].head()

,ID,날짜,자동분류1,자동분류2,자동분류3,URL,언론사,원문,번역문
0,20961563,2018-10-01 00:00:00,"경제,국제경제","IT_과학,IT_과학일반","문화,문화일반",http://www.sedaily.com/NewsView/1S5QRSR5T2,서울경제,스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.,Skinner's reward is mostly eye-watering.
1,20930897,2018-09-14 00:00:00,"IT_과학,IT_과학일반",NaN,NaN,http://www.sedaily.com/NewsView/1S4MSFM0IH,서울경제,심지어 어떤 문제가 발생할 건지도 어느 정도 예측이 가능하다.,Even some problems can be predicted.
2,20307875,2019-04-19 00:00:00,"IT_과학,과학",NaN,NaN,http://news.kmib.co.kr/article/view.asp?arcid=...,국민일보,오직 하나님만이 그 이유를 제대로 알 수 있을 겁니다.,Only God will exactly know why.
3,20314977,2019-04-27 00:00:00,"국제,중국","경제,국제경제",NaN,http://news.kmib.co.kr/article/view.asp?arcid=...,국민일보,중국의 논쟁을 보며 간과해선 안 될 게 기업들의 고충이다.,Businesses should not overlook China's dispute.
4,20268904,2019-03-13 00:00:00,"문화,방송_연예","사회,미디어",NaN,http://news.kmib.co.kr/article/view.asp?arcid=...,국민일보,박자가 느린 노래는 오랜 시간이 지나 뜨는 경우가 있다.,Slow-beating songs often float over time.


In [7]:
for i, df in enumerate(train_dfs):
    print(f"DF#{i}:", df.shape)
    print(df.columns)

DF#0: (200011, 9)
Index(['ID', '날짜', '자동분류1', '자동분류2', '자동분류3', 'URL', '언론사', '원문', '번역문'], dtype='object')
DF#1: (200541, 9)
Index(['ID', '날짜', '자동분류1', '자동분류2', '자동분류3', 'URL', '언론사', '원문', '번역문'], dtype='object')
DF#2: (200835, 9)
Index(['ID', '날짜', '자동분류1', '자동분류2', '자동분류3', 'URL', '언론사', '원문', '번역문'], dtype='object')
DF#3: (200000, 9)
Index(['ID', '날짜', '자동분류1', '자동분류2', '자동분류3', 'URL', '언론사', '원문', '번역문'], dtype='object')


In [8]:
total_df = pd.concat(train_dfs)
total_df.head()

,ID,날짜,자동분류1,자동분류2,자동분류3,URL,언론사,원문,번역문
0,20961563,2018-10-01 00:00:00,"경제,국제경제","IT_과학,IT_과학일반","문화,문화일반",http://www.sedaily.com/NewsView/1S5QRSR5T2,서울경제,스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.,Skinner's reward is mostly eye-watering.
1,20930897,2018-09-14 00:00:00,"IT_과학,IT_과학일반",NaN,NaN,http://www.sedaily.com/NewsView/1S4MSFM0IH,서울경제,심지어 어떤 문제가 발생할 건지도 어느 정도 예측이 가능하다.,Even some problems can be predicted.
2,20307875,2019-04-19 00:00:00,"IT_과학,과학",NaN,NaN,http://news.kmib.co.kr/article/view.asp?arcid=...,국민일보,오직 하나님만이 그 이유를 제대로 알 수 있을 겁니다.,Only God will exactly know why.
3,20314977,2019-04-27 00:00:00,"국제,중국","경제,국제경제",NaN,http://news.kmib.co.kr/article/view.asp?arcid=...,국민일보,중국의 논쟁을 보며 간과해선 안 될 게 기업들의 고충이다.,Businesses should not overlook China's dispute.
4,20268904,2019-03-13 00:00:00,"문화,방송_연예","사회,미디어",NaN,http://news.kmib.co.kr/article/view.asp?arcid=...,국민일보,박자가 느린 노래는 오랜 시간이 지나 뜨는 경우가 있다.,Slow-beating songs often float over time.


In [9]:
total_df.tail()

,ID,날짜,자동분류1,자동분류2,자동분류3,URL,언론사,원문,번역문
199995,388960,20180904,"국제,미국_북미",NaN,NaN,http://news.kmib.co.kr/article/view.asp?arcid=...,국민일보,복무기간 단축안은 10월 전역자부터 2주 단위로 하루씩 단축해 육군·해병대·의무경찰...,The proposed reduction of the service period w...
199996,3299067,20180619,"사회,사회일반","정치,외교","정치,청와대",http://www.sedaily.com/NewsView/1S0VBEJFH0,서울경제,"실제로 이번 인사에서 인천지검 특수부장, 서울중앙지검 증권범죄합동수사단장 등을 지낸...","In fact, the vice chief of the Seoul Eastern D..."
199997,364266,20180829,"IT_과학,과학","경제,경제일반",NaN,http://news.kmib.co.kr/article/view.asp?arcid=...,국민일보,29일 서울 서초구 한국산업기술진흥협회 중회의실에서 열린 ‘이공계 우수인재 양성 및...,"On the 29th, at a meeting of experts on ""how t..."
199998,9684562,20190304,"사회,교육_시험","지역,지역일반",NaN,NaN,노컷뉴스,광주시교육청은 “지난 1일과 2일 한유총 광주지회로부터 장휘국 교육감 면담 요청이 ...,"The Gwangju Office of Education said, “There w..."
199999,3054292,20190523,"정치,청와대","정치,외교","정치,북한",http://www.hani.co.kr/arti/politics/bluehouse/...,한겨레,"과학기술정보통신부 과학기술혁신본부장에는 김성수(58) 한국화학연구원장, 행정안전부 ...",The appointment for the head of the Science an...


In [10]:
total_df.shape

(801387, 9)

In [11]:
total_df.to_excel(merged_train_file)